# In this Notebook I want to see the difference the anealing schedule makes

In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy as sp
%matplotlib inline

In [ ]:
# here I am loading in the annealing schedule from the vesuvius device
SAB = mylib.load_project_data('ISI_A&B.txt')
ss = SAB[:,0]
AB = np.transpose(SAB[:,1:])

# plot the annealing schedule from vesuvius
plt.figure()
plt.plot(ss,AB[0],label = 'A(s)')
plt.plot(ss,AB[1],label = 'B(s)')
plt.xlabel('s')
plt.ylabel('A(s) and B(s)')
plt.legend(framealpha =0, loc = 'best')
plt.show()

Above is the annealing schedule from the ~512 qubit D-Wave 2 (Vesuvius) device (specifically the one previously at the University of Southern California). This is actually used experimentally. In order to make this more general I will need to be able to interpolate this onto the points I want. Based on a quick look at the data I have decided that linear interpolation will be good enough, no need for cubic.

In [ ]:
# In this cell I make a function to interpolate the data from the schedule above onto any arbitrary s array

def interpolate_exper(exp_ss,exp_AB,points):
    '''This function takes the experimental AB you feed in which are discrete and interpolates them onto a new set of points'''
    new_ss = np.linspace(0,1,points)
    new_AB = np.zeros((2,points))
    new_AB[0] = sp.interp(new_ss, exp_ss, exp_AB[0])
    new_AB[1] = sp.interp(new_ss, exp_ss, exp_AB[1])
    return new_ss, new_AB

points = len(ss)*4
new_ss,new_AB = interpolate_exper(ss,AB,points)

plt.figure(figsize = (10,7))
plt.title('Linear Interpolation of the Annealing Schedule')
plt.plot(new_ss, new_AB[0],label = '$A_{interp}(s)$',color = 'blue') 
plt.plot(new_ss, new_AB[1],label = '$B_{interp}(s)$',color = 'red') 
plt.plot(ss[::15], AB[0][::15], 'o', label = '$A_{exp}(s)$', color = 'blue') 
plt.plot(ss[::15], AB[1][::15],'o',label = '$B_{exp}(s)$',color = 'red') 
plt.xlabel('s')
plt.legend(framealpha =0, loc = 'best')
plt.show()

In [ ]:
def vesuvius_schedule(points):
    '''This function returns the annealing schedule which is an interpolation of that used in the ~512 qubit D-Wave 2 (Vesuvius) 
    device (specifically the one previously at the University of Southern California, although different devices from the 
    same generation have nearly identical schedules)'''
    ves_SAB = mylib.load_project_data('ISI_A&B.txt') # this is the saved text file of the schedule used in the vesuvius
    ves_ss = SAB[:,0]
    ves_AB = np.transpose(SAB[:,1:])
    interp_ss, interp_AB = interpolate_exper(ves_ss,ves_AB,points)
    interp_AB[0] =  interp_AB[0]/ interp_AB[0][0]
    interp_AB[1] =  interp_AB[1]/ interp_AB[1][-1]
    return interp_AB

Next I want to see how this schedule affects the accuracy of the final solution to the problem. I will plot the annealing schedule, energy gap, probability of being in the instanteous ground state for this annealing schedule and a linear one. Then print final probability of being in the ground state at the end

In [ ]:
def exponential_schedule(points):
    '''A(s) is indexed with return[0] B(s) is indexed with return[1]'''
    a = 5.; b = 5.
    return np.asarray([np.exp(-a*np.linspace(0,1,points)),np.exp(-b*(-np.linspace(0,1,points)+1))])

ss=np.linspace(0,1,1000)
AB = exponential_schedule(1000)
plt.figure()
plt.plot(ss, AB[0],label = '$A(s)$',color = 'blue') 
plt.plot(ss, AB[1],label = '$B(s)$',color = 'red') 
plt.xlabel('s')
plt.legend(framealpha = 0, loc = 'best')
plt.show()

In [ ]:
# here I want to develop the function that will make a schedule from the energy gap
points = 1000
delta_eigenvals = -np.sin(np.linspace(0,np.pi,int(points/10)))+1.1

def inverse_energy_schedule(points):
        P = np.linspace(0,1,int(points/10))
        inverse_delta_eigenvals = np.asarray(delta_eigenvals).astype(float)**-1
        schedule = np.cumsum(inverse_delta_eigenvals/sum(inverse_delta_eigenvals))
        B = sp.interp(np.linspace(0,1,points), schedule, P)
        return B

def alternate_inverse(points):
        P = np.linspace(0,1,int(points/10))
        sum_delta_eigenvals = np.sum(delta_eigenvals)
        schedule = np.cumsum(sum_delta_eigenvals)/sum_delta_eigenvals
        return schedule
    
B = inverse_energy_schedule(points)
B_alt = alternate_inverse(points)
ss = np.linspace(0,1,points)
plt.figure()
plt.plot(ss,B,label = '$B(s)$')
plt.plot(ss,B,label = '$B_{alt}(s)$')
plt.plot(ss[::10],delta_eigenvals,label = '$\Delta \lambda$')
plt.legend(framealpha = 0, loc = 'best')
plt.xlabel('s')
plt.show()


I used the code above  to construct the 'natural' annealing schedule. Where what I have done above would be the B parameter

## Linear Schedule

In [ ]:
h = [1.2,3.2,2.9]
J = [[1.,2.2],[2.3]]
qubits = 3
a = mylib.Anneal(3,[h,J],T=5,points = 10000)
a.run()
a.show_results()

## Vesuvius Schedule

In [ ]:
h = [1.2,3.2,2.9]
J = [[1.,2.2],[2.3]]
qubits = 3
a = mylib.Anneal(3,[h,J],T=10000,points = 10000,sched_func = mylib.vesuvius_schedule)
a.run()
a.show_results()

## Exponential Schedule

In [ ]:
h = [1.2,3.2,2.9]
J = [[1.,2.2],[2.3]]
qubits = 3
a = mylib.Anneal(3,[h,J],T=10000,points = 10000,sched_func = mylib.exponential_schedule_ab(4.,4.))
a.run()
a.show_results()

## Inverse Energy Schedule

In [ ]:
h = [1.2,3.2,2.9]
J = [[1.,2.2],[2.3]]
qubits = 3
a = mylib.Anneal(3,[h,J],T=10000,points = 10000,sched_func = 'natural')
a.run()
a.show_results()

In [ ]:
schedfuncname = ['linear','exponential fast','exponential medium','exponential slow','inverse energy','vesuvius']
schedfunc = [mylib.linear_schedule, mylib.exponential_schedule_ab(10,10), mylib.exponential_schedule_ab(5,5),
             mylib.exponential_schedule_ab(1,1), 'natural', mylib.vesuvius_schedule]

h = [1.2,3.2,2.9]
J = [[1.,2.2],[2.3]]
qubits = 3
last_prob = []
an = []

for i,f in enumerate(schedfunc):
    print '\n',schedfuncname[i]
    an_n= mylib.Anneal(qubits,[h,J],T=5,points = 10000,sched_func = f)
    an_n.run()
    an.append(an_n)
    last_prob.append(an_n.problem_x0_prob[-1]) 


In [ ]:
import tabulate
print '\n Probability to be found in the correct final state for different annealing schedules\n'

schedfuncname = ['linear','exponential fast','exponential medium','exponential slow','inverse energy','vesuvius']
last_prob =  []
for i in range(len(an)):
    last_prob.append(an[i].problem_x0_prob[-1]) 

headers = ['Schedule','Probability of being in correct state']
table = []

lpcop = list(np.copy(last_prob))
nam = list(np.copy(schedfuncname))

for i in range(len(schedfuncname)):
    table.append([nam.pop(np.argmax(lpcop)),lpcop.pop(np.argmax(lpcop))])
print tabulate.tabulate(table, headers, tablefmt="grid",floatfmt=".8f")

In [ ]:
plt.figure(figsize = (8,20))
ax = []
axtwin = []
for i in range(len(an)):
    if i == 0: ax.append(plt.subplot2grid((len(an),1), (i,0)))
    else: ax.append(plt.subplot2grid((len(an),1), (i,0), sharex = ax[0]))
    ax[i].set_ylabel('$\Delta \lambda$',color = 'black')
    if i == len(an)-1: ax[i].set_xlabel('s')
    axtwin.append(ax[i].twinx())
    axtwin[i].set_ylabel('A(s), B(s)')
    axtwin[i].text(0.5,0.9,schedfuncname[i],verticalalignment='center', horizontalalignment='center')
    axtwin[i].plot(an[i].ss, an[i].AB[0], label = '$A(s)$',color = 'red')
    axtwin[i].plot(an[i].ss, an[i].AB[1], label = '$B(s)$', color = 'blue')
    ax[i].plot(an[i].ss,an[i].delta_eigenvals,color='black',label = '$\Delta \lambda$')
plt.show()